In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from numpy.linalg import inv, norm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
boston = load_boston()
df_boston = pd.DataFrame(boston.data, columns=boston.feature_names)
df_boston['MEDV'] =  boston.target

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [ ]:
X = df_boston[['PTRATIO', 'LSTAT']]
Y = df_boston.MEDV
X = (X - X.mean(axis=0))/ X.std(axis=0)
X, Y = X.to_numpy(), Y.to_numpy().reshape(-1, 1)
N, F = X.shape
X = np.hstack((np.ones((N, 1)), X))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
n_test = x_test.shape[0]

In [ ]:
n_epochs = 190
alpha = 0.01
def stochastic_gradient_descent(X, Y, F, alpha, n_epochs):
    W = np.zeros((F+1, 1))
    N = X.shape[0]
    loss = []
    for _ in range(n_epochs):
        np.random.shuffle(X)
        sum_errors = 0
        for i in range(N):
            grad = 2/N * np.matmul((np.matmul(X[i:i+1,:], W) - Y[i:i+1,:]).T, X[i:i+1,:])
            W = W - alpha*grad.T
            err = norm(np.matmul(X[i:i+1,:],W) - Y[i:i+1,:])**2
            sum_errors += err
            
        loss.append(1/N * sum_errors)
 
    return W, loss

W_sgd, loss_sgd = stochastic_gradient_descent(x_train, y_train, F, alpha, n_epochs)

test_loss = np.sqrt(1/n_test * norm(np.matmul(x_test, W_sgd) - y_test)**2)
print("The RMSE for SGD is:", test_loss)

The RMSE for SGD is: 8.584060278266891


In [ ]:
n_epochs = 190
alpha = 0.01
mb = 32
def minibatch_gradient_descent(X, Y, F, alpha, n_epochs, mb):
    W = np.zeros((F+1, 1))
    N = X.shape[0]
    loss = []
    for _ in range(n_epochs):
        np.random.shuffle(X)
        sum_errors = 0
        for i in range(N-mb+1):
            grad = 2/N * np.matmul((np.matmul(X[i:i+mb,:], W) - Y[i:i+mb,:]).T, X[i:i+mb,:])
            err = 1/mb * norm(np.matmul(X[i:i+mb,:],W) - Y[i:i+mb,:])**2
            W = W - alpha*grad.T
            sum_errors += err

        loss.append(1/N * sum_errors)
 
    return W, loss
W_mbgd, loss_mbgd = minibatch_gradient_descent(x_train, y_train, F, alpha, n_epochs, mb)

test_loss = np.sqrt(1/n_test * norm(np.matmul(x_test, W_mbgd) - y_test)**2)
print("The RMSE loss for minibatch gradient descent is:", test_loss)

The RMSE loss for minibatch gradient descent is: 8.557951393972616


In [ ]:
clf_ = SGDRegressor(max_iter=190, alpha=0.01)
clf_.fit(x_train, y_train)
y_pred_sksgd=clf_.predict(x_test)
print('RMSE for SGDRegressor is:',np.sqrt(mean_squared_error(y_test, y_pred_sksgd)))

RMSE for SGDRegressor is: 8.612299152021352


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
